In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from tqdm import tqdm
import h5py 
import os
import glob
import utils
os.environ['CUDA_VISIBLE_DEVICES']='1'

tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True).to('cuda')
model.eval();

/home/ztang/.conda/envs/torch_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/home/ztang/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/1d020b803b871a976f5f3d5565f0eac8f2c7bb81/bert_layers.py:125: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of the model checkpoint at zhihan1996/DNABERT-2-117M were not used when initializing BertM

## Lenti-MPRA

In [ ]:
celltype = 'K562'
file = h5py.File('/home/ztang/multitask_RNA/data/lenti_MPRA/'+celltype+'_data.h5','r')

In [ ]:
hidden_output = h5py.File('../data/lenti_MPRA_embed/dnabert2_'+celltype+'.h5','w')
batch_size = 128
output_cache = []
for i in tqdm(range(0,len(file['seq']),batch_size)):
    seq = file['seq'][i:i+batch_size].astype('U230')
    input_ids = tokenizer(seq.tolist(), return_tensors="pt",padding='max_length',max_length=58)["input_ids"].to('cuda')
    hidden_states = model(input_ids)[0].cpu().detach().numpy() # [1, sequence_length, 768]
    output_cache.extend(hidden_states)
hidden_output.create_dataset(name='seq',data = np.array(output_cache))
hidden_output.create_dataset(name='mean',data = file['mean'][:])
hidden_output.close()

## Chip/Clip seq data

In [2]:
max_len = 0
file_list = glob.glob('../data/chip/*.h5')
for file in file_list:
    tf_name = file.split('/')[-1][:-7]
    bert_output = h5py.File('../data/chip/DNABERT/'+tf_name+'_200.h5','w')
    batch_size = 128
    file = h5py.File(file,'r')
    for label in ('train','valid','test'):
        output_cache = []  
        for i in tqdm(range(0,len(file['x_'+label]),batch_size)):
            seq = file['x_'+label][i:i+batch_size].astype('int')
            seq = np.transpose(seq,(0,2,1))
            seq = utils.onehot_to_seq(seq)
            input_ids = tokenizer(seq, return_tensors="pt",padding='max_length',max_length=57)["input_ids"].to('cuda')
            hidden_states = model(input_ids)[0].cpu().detach().numpy() # [1, sequence_length, 768]
            output_cache.extend(hidden_states)
        bert_output.create_dataset(name='x_'+label,data = np.array(output_cache),dtype = 'float32')
        bert_output.create_dataset(name='y_'+label,data = file['y_'+label][:],dtype='int') 
    bert_output.close()


100%|██████████| 57/57 [00:07<00:00,  7.80it/s]


In [8]:
max_len = 0
file_list = glob.glob('../data/eclip/*.h5')
for file in file_list:
    tf_name = file.split('/')[-1][:-7]
    bert_output = h5py.File('../data/eclip/DNABERT/'+tf_name+'_200.h5','w')
    batch_size = 128
    file = h5py.File(file,'r')
    for label in ('train','valid','test'):
        output_cache = []  
        for i in tqdm(range(0,len(file['X_'+label]),batch_size)):
            seq = file['X_'+label][i:i+batch_size].astype('int')
            seq = np.transpose(seq,(0,2,1))
            seq = utils.onehot_to_seq(seq)
            input_ids = tokenizer(seq, return_tensors="pt",padding='max_length',max_length=53)["input_ids"].to('cuda')
            hidden_states = model(input_ids)[0].cpu().detach().numpy() # [1, sequence_length, 768]
            output_cache.extend(hidden_states)
        bert_output.create_dataset(name='x_'+label,data = np.array(output_cache),dtype = 'float32')
        bert_output.create_dataset(name='y_'+label,data = file['Y_'+label][:],dtype='int') 
    bert_output.close()


  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00,  9.46it/s]


## MT Splice data

In [5]:
file = h5py.File('../data/alternative_splicing/delta_logit.h5','r')
bert_output = h5py.File('../data/alternative_splicing/DNABERT_splice.h5','w')
batch_size = 32
max_len = 0
for label in ('valid','test','train'):
    l_cache = []
    r_cache = [] 
    for i in tqdm(range(0,len(file['x_'+label]),batch_size)):
        seq = file['x_'+label][i:i+batch_size].astype('int')
        seq = utils.onehot_to_seq(seq)
        clean_seq = seq
        #clean_seq = [s if 'N' not in s else s.replace('N','[PAD]') for s in seq ]
        l_seq = []
        r_seq = []
        for seq in clean_seq:
            l_seq.append(seq[:400])
            r_seq.append(seq[400:])
        l_input = tokenizer(l_seq, return_tensors="pt",padding='max_length',max_length=270)["input_ids"].to('cuda')
        r_input = tokenizer(r_seq, return_tensors="pt",padding='max_length',max_length=270)["input_ids"].to('cuda')

        l_output = model(l_input)[0].cpu().detach().numpy() # [1, sequence_length, 768]
        r_output = model(r_input)[0].cpu().detach().numpy() # [1, sequence_length, 768]
        l_cache.extend(l_output)
        r_cache.extend(r_output)
    bert_output.create_dataset(name='xl_'+label,data = np.array(l_cache),dtype = 'float32')
    bert_output.create_dataset(name='xr_'+label,data = np.array(r_cache),dtype = 'float32')
    bert_output.create_dataset(name='y_'+label,data = file['y_'+label][:],dtype='float32') 


100%|██████████| 1189/1189 [13:03<00:00,  1.52it/s]


In [6]:
for key in bert_output.keys():
    print(bert_output[key].shape)

bert_output.close()

(11840, 270, 768)
(38028, 270, 768)
(1088, 270, 768)
(11840, 270, 768)
(38028, 270, 768)
(1088, 270, 768)
(11840, 56, 2)
(38028, 56, 2)
(1088, 56, 2)


## RNA-enlong data

In [2]:
bert_output = h5py.File('../data/RNAenlong/dnabert_embed.h5','w')
file = h5py.File('../data/RNAenlong/insert_dataset.h5','r')
batch_size = 32
for dataset in ['test','train','valid']:
    key = 'X_'+dataset
    onehot = file[key]
    string_seq = utils.onehot_to_seq(onehot)
    token_seq = tokenizer(string_seq, return_tensors="pt",padding='max_length',max_length=46)["input_ids"].to('cuda')
    output_cache = []
    for seq_i in tqdm(range(0,len(token_seq),batch_size)):
        hidden_states = model(token_seq[seq_i:seq_i+batch_size])[0].cpu().detach().numpy() # [1, sequence_length, 768]
        output_cache.extend(hidden_states)
    bert_output.create_dataset(name=key,data = np.array(output_cache))
    bert_output.create_dataset(name='Y_'+dataset,data = file['Y_'+dataset][:])

100%|██████████| 36/36 [00:01<00:00, 34.37it/s]


In [3]:
for key in bert_output.keys():
    print(bert_output[key].shape)

bert_output.close()

(1137, 46, 768)
(9131, 46, 768)
(1149, 46, 768)
(1137, 3)
(9131, 3)
(1149, 3)
